Implementation of questions 7-10 for Homework #1 from "Learning from Data" / Professor Yaser Abu-Mostafa, Caltech
http://work.caltech.edu/homework/hw1.pdf

This notebook contains features a functional form of the solution. It includes all necessary functions
and a line class in the first code cell (below this markdown cell). The second code cell then executes
the perceptron learning algorithm and displays the obtained results.

Date: 11/02/2021
Author: Deaga

In [7]:
import random
import numpy as np 

def random_point(xlim=[-1,1],ylim=[-1,1]):
    """
    Creates a random point with coordinates(x,y)
    Random values will be bounded by xlim and ylim
    """

    x = random.uniform(xlim[0],xlim[1])
    y = random.uniform(ylim[0],ylim[1])

    return (x,y)

class line:

    def __init__(self,p1=None,p2=None,angular=None,linear=None,random=False,xlim=[-1,1],ylim=[-1,1]):
        """
        Initialize a line from either a pair of points (x,y)
        or from angular and linear coefficients.

        If random is set to true, generate a line from two random points,
        bounded by xlim and ylim.
        """

        if random:
            #Get a random line
            self.random_line(xlim,ylim)
        
        else:
            #Use the redefine function with the given inputs
            self.redefine(p1=p1,p2=p2,angular=angular,linear=linear)
    
    def redefine(self,p1=None,p2=None,angular=None,linear=None):
        """
        Redefines current line from either a pair of points (x,y)
        or from angular and linear coefficients.
        """

        #First case: given two points p1 and p2
        if (p1 != None and p2 != None):
            try:
                self.a= (p1[1] - p2[1]) / (p1[0] - p2[0]) #a = (y1-y2)/(x1-x2)
                self.b= (p1[0]*p2[1] - p2[0]*p1[1]) / (p1[0] - p2[0]) #b = (x1y2 - x2y1)/(x1 - x2)
            except:
                raise ValueError('Invalid format for p1 and/or p2. Use tuples with just two entries, p1=(x1,y1) and p2=(x2,y2).')
        #With given angular and linear
        elif (angular != None and linear != None):
            try:
                self.a=angular
                self.b=linear
            except:
                raise ValueError('Invalid format for angular or linear. Use numbers as input!')
        else:
            raise ValueError('Invalid inputs! p1 and p2 must be tuples in the form p1=(x1,y1), p2=(x2,y2).\nOtherwise, use angular=number and linear=number!')
    
    def random_line(self,xlim=[-1,1],ylim=[-1,1]):
        """
        Returns a line that passes two random points.
        Both points will be in the domain [xlim] x [ylim]
        """

        p1 = random_point(xlim,ylim)
        p2 = random_point(xlim,ylim)

        self.redefine(p1=p1,p2=p2)

    def get_y(self,x=0):
        """
        Calculates y value for a given x, for the current line.
        """

        return self.a*x+self.b

    def get_x(self,y=0):
        """
        Calculates x value for a given y, for the current line
        """

        return (y-self.b)/self.a

    def map(self,xp,yp):
        """
        Maps a value of +1 or -1 to point defined by p=(xp,yp)
        If yp > y(xp), return +1
        Else, return -1
        """
        if yp > self.get_y(xp):
            return 1
        else:
            return -1

def run_experiment(xlim=[-1,1],ylim=[-1,1]):
    """
    Single run of the experiment for HW1:
    - Create a random line based on two random points: yl = al*x + b
    - Create a random point. p=(xp,yp)
    - If yp > yl(xp) (point above line), return 1
    - Else, return -1
    """

    #Create line and point
    rline = line(random=True,xlim=xlim,ylim=ylim)
    rpoint = random_point(xlim=xlim,ylim=ylim)

    #Return mapping function
    return rline.map(rpoint)

def h_func(weights,point):
    """
    Returns h(point) = sign(sum(w_i*coord_i))
    weights and point must be lists/tuples/iterables of the same length
    """

    #Initialize sum as 0
    sum = 0


    for w,coord in zip(weights,point):
        sum += w*coord
    
    return np.sign(sum)

In [8]:
#Initialize the problem

#Number of weights and coordinates, besides the artificial weight and coordinate x0 and w0
d=2

#Add 1 to account for artificial weight and coordinate
d += 1

#Coordinate limits for x and y
xlim=[-1,1]
ylim=[-1,1]

#Number of points from the data set
N = 10

#Define maximum number of iterations per experiment
max_iter=1000

#Create empty list to store how many iterations it took in each experiment
actual_iter=[] 

#How many experiments to run?
N_exp=1000

#List with the probability of error for each run
error_probability=[]

for exp in range(0,N_exp):

    #Display current experiment in console, every 20 experiments
    if (exp+1 == 1) or ((exp+1) % 50 ==0):
        print(f'Running experiment number {exp+1}...')
    
    # Initiate iteration counter
    iter_count=0

    #Target line
    target_line = line(random=True)
    target_function = target_line.map

    #Initialize lists for w (weights), p (points) and h (perceptron function)
    w = [0 for i in range(0,d)] #As many as there are coordinates

    h = [0 for i in range(0,N)] #As many as there are points
    p = [] #Start empty

    #Initialize random points
    for i in range(0,len(h)):
        #Get x and y coordinates for random points
        x,y = random_point()

        #Add them to p list. Each entry is a tuple of form (1,x,y). 1 is the artificial coordinate
        p.append((1,x,y))

    #Start iterating weights
    while True:
        #Store old weight values to check convergence
        w_old = w.copy()
        
        #Increment iteration counter
        iter_count += 1

        #Iterate through points
        for i in range(0,len(h)):
            h[i]=np.sign(np.inner(w,p[i]))
            # hval=np.inner(w,point)

        #Check for a wrong point
        for hval,point in zip(h,p):
            target = target_function(point[1],point[2])

            if hval != target:
            
                #Update weights
                for i in range(0,len(w)):
                    w[i]=w[i]+target*point[i]
                
                #Exit loop
                break

        #Check convergence
        if ((np.array_equal(w,w_old)) or iter_count > max_iter):
            actual_iter.append(iter_count)
            break

    
    #Test perceptron on N random points
    error_count=0
    error_N = 10*N  #How many points to test for errors
    for i in range(0,error_N):
        x_test,y_test = random_point()
        h_test = np.sign(np.inner(w,[1,x_test,y_test]))
        h_actual = target_function(x_test,y_test)

        if h_test != h_actual:
            error_count += 1

    #Append to error_probability
    error_probability.append(error_count/error_N)

#Print results
print(f'\nAverage number of iterations to converge: {int(round(np.average(actual_iter),0))}')
print(f'Average error probability: {round(np.average(error_probability)*100,2)}%')

Running experiment number 1...
Running experiment number 50...
Running experiment number 100...
Running experiment number 150...
Running experiment number 200...
Running experiment number 250...
Running experiment number 300...
Running experiment number 350...
Running experiment number 400...
Running experiment number 450...
Running experiment number 500...
Running experiment number 550...
Running experiment number 600...
Running experiment number 650...
Running experiment number 700...
Running experiment number 750...
Running experiment number 800...
Running experiment number 850...
Running experiment number 900...
Running experiment number 950...
Running experiment number 1000...

Average number of iterations to converge: 14
Average error probability: 11.23%
